In [1]:
# notebook of the audio pattern recognition project
# Marco Nicolini
import tensorflow as tf
import os
import numpy as np 
import pandas as pd 
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import glob 
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import sys

CREMA = 'Crema/'
TESS = 'Tess/'
RAV = 'Ravdess/'
SAVEE = 'Savee/'
EMOVO = 'EMOVO/'
EMODB = 'EmoDB/'
SHEMO = 'ShEMO/'
URDU = 'URDU/'
JL = 'JLcorpus/'
AESDD = 'AESDD/'

<h3>CREMA-D dataset</h3>

<a href="https://github.com/CheyneyComputerScience/CREMA-D">CREMA-D link</a>

In [2]:
dir_list = os.listdir(CREMA)
dir_list.sort()
gender = []
emotion = []
path = []
female = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]

for i in dir_list: 
    part = i.split('_')
    if int(part[0]) in female:
        temp = 'female'
    else:
        temp = 'male'
    gender.append(temp)
    if part[2] == 'SAD':
        emotion.append('sad')
    elif part[2] == 'ANG':
        emotion.append('angry')
    elif part[2] == 'DIS':
        emotion.append('disgust')
    elif part[2] == 'FEA':
        emotion.append('fear')
    elif part[2] == 'HAP':
        emotion.append('happy')
    elif part[2] == 'NEU':
        emotion.append('neutral')
    else:
        emotion.append('Unknown')
    path.append(CREMA + i)
    
CREMA_df = pd.DataFrame(emotion, columns = ['labels'])
CREMA_df = pd.concat([CREMA_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
CREMA_df['source'] = 'CREMA'
CREMA_df = pd.concat([CREMA_df,pd.DataFrame(path, columns = ['path'])],axis=1)
CREMA_df.labels.value_counts()

angry      1271
disgust    1271
fear       1271
happy      1271
sad        1271
neutral    1087
Name: labels, dtype: int64

<h3>SAVEE dataset</h3>

<a href="http://kahlan.eps.surrey.ac.uk/savee/">SAVEE link</a>

In [3]:
# Get the data location for SAVEE
dir_list = os.listdir(SAVEE)

# parse the filename to get the emotions
emotion=[]
path = []
gender = []
for i in dir_list:
    gender.append('male')
    if i[-8:-6]=='_a':
        emotion.append('angry')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
    elif i[-8:-6]=='sa':
        emotion.append('sad')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
    else:
        emotion.append('error') 
    path.append(SAVEE + i)
    
# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_df = pd.concat([SAVEE_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
SAVEE_df['source'] = 'SAVEE'
SAVEE_df = pd.concat([SAVEE_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.labels.value_counts()

neutral     120
sad          60
surprise     60
fear         60
disgust      60
happy        60
angry        60
Name: labels, dtype: int64

<h3>RAVDESS dataset</h3>

<a href="https://zenodo.org/record/1188976#.YbMujC1abOQ">RAVDESS link</a>

In [4]:
dir_list = os.listdir(RAV)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
new_dir_list = []
for i in dir_list:
    new_dir_list.append(RAV + i)

emotion = []
gender = []
path = []
for i in new_dir_list:
    fname = os.listdir(i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(i + '/' + f)

     


RAV_df = pd.DataFrame(emotion, columns = ['labels'])
RAV_df = RAV_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([RAV_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
RAV_df['source'] = 'RAV'
RAV_df = pd.concat([RAV_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
RAV_df.labels.value_counts()

neutral     288
surprise    192
angry       192
fear        192
disgust     192
sad         192
happy       192
Name: labels, dtype: int64

<h3>TESS dataset</h3>

<a href="https://doi.org/10.5683/SP2/E8H2MF">TESS link</a>

In [5]:
dir_list = os.listdir(TESS)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
new_dir_list = []

path = []
emotion = []
gender = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        gender.append("female")
        if i == 'OAF_angry' or i == 'YAF_angry':
            emotion.append('angry')
        elif i == 'OAF_disgust' or i == 'YAF_disgust':
            emotion.append('disgust')
        elif i == 'OAF_Fear' or i == 'YAF_fear':
            emotion.append('fear')
        elif i == 'OAF_happy' or i == 'YAF_happy':
            emotion.append('happy')
        elif i == 'OAF_neutral' or i == 'YAF_neutral':
            emotion.append('neutral')                                
        elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
            emotion.append('surprise')               
        elif i == 'OAF_Sad' or i == 'YAF_sad':
            emotion.append('sad')
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['labels'])
TESS_df = pd.concat([TESS_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
TESS_df['source'] = 'TESS'
TESS_df = pd.concat([TESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.labels.value_counts()

fear        400
surprise    400
sad         400
angry       400
disgust     400
happy       400
neutral     400
Name: labels, dtype: int64

<h3>EMOVO dataset</h3>

<a href="https://art.torvergata.it/handle/2108/97109">EMOVO link</a>

In [6]:
dir_list = os.listdir(EMOVO)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
new_dir_list = []
for i in dir_list:
    new_dir_list.append(EMOVO + i)
new_dir_list

emotion = []
gender = []
path = []
for i in new_dir_list:
    fname = os.listdir(i)
    for f in fname:
        part = f.split('-')
        emotion.append(part[0])
        temp = part[1]
        if temp[0] == 'f':
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(i + '/' + f)
        
EMOVO_df = pd.DataFrame(emotion, columns = ['labels'])
EMOVO_df = EMOVO_df.replace({'neu':'neutral', 'gio':'happy', 'tri':'sad', 'rab':'angry', 'pau':'fear', 'dis':'disgust', 'sor':'surprise'})
EMOVO_df = pd.concat([EMOVO_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
EMOVO_df['source'] = 'EMOVO'  
EMOVO_df = pd.concat([EMOVO_df,pd.DataFrame(path, columns = ['path'])],axis=1)
EMOVO_df.labels.value_counts()

surprise    84
neutral     84
sad         84
fear        84
disgust     84
happy       84
angry       84
Name: labels, dtype: int64

<h3>EmoDB dataset</h3>

<a href="http://emodb.bilderbar.info/index-1280.html">EmoDB link</a>

In [7]:
dir_list = os.listdir(EMODB)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
emotion = []
gender = []
path = []

for f in dir_list:
    # elimino L -> bored
    if f[5] == "L": continue
    emotion.append(f[5])
    temp = f[0:2]
    if temp in ["03", "10", "11", "12", "15"]:
        temp = "male"
    else:
        temp = "female"
    gender.append(temp)
    path.append(EMODB + f)

EMODB_df = pd.DataFrame(emotion, columns = ['labels'])
EMODB_df = EMODB_df.replace({'N':'neutral', 'F':'happy', 'T':'sad', 'W':'angry', 'A':'fear', 'E':'disgust'})
EMODB_df = pd.concat([EMODB_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
EMODB_df['source'] = 'EmoDB' 
EMODB_df = pd.concat([EMODB_df,pd.DataFrame(path, columns = ['path'])],axis=1)
EMODB_df.labels.value_counts()

angry      127
neutral     79
happy       71
fear        69
sad         62
disgust     46
Name: labels, dtype: int64

<h3>ShEMO dataset</h3>

<a href="https://link.springer.com/article/10.1007/s10579-018-9427-x">ShEMO link</a>

In [8]:
dir_list = os.listdir(SHEMO + "female")
dir_list += os.listdir(SHEMO + "male")
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
emotion = []
gender = []
path = []
for f in dir_list:
    st_path = SHEMO
    emotion.append(f[3])
    temp = f[0]
    if temp == "M":
        temp = "male"
        st_path += "male/" + f
    else:
        temp = "female"
        st_path += "female/" + f
    gender.append(temp)
    path.append(st_path)
    
SHEMO_df = pd.DataFrame(emotion, columns = ['labels'])
SHEMO_df = SHEMO_df.replace({'N':'neutral', 'H':'happy', 'S':'sad', 'A':'angry', 'F':'fear', 'W':'surprise'})
SHEMO_df = pd.concat([SHEMO_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
SHEMO_df['source'] = 'ShEMO'  
SHEMO_df = pd.concat([SHEMO_df,pd.DataFrame(path, columns = ['path'])],axis=1)
SHEMO_df.labels.value_counts()

angry       1059
neutral     1028
sad          449
surprise     225
happy        201
fear          38
Name: labels, dtype: int64

<h3>URDU dataset</h3>

<a href="https://github.com/siddiquelatif/URDU-Dataset">URDU link</a>

In [9]:
dir_list = os.listdir(URDU)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]
new_dir_list = []
for i in dir_list:
    new_dir_list.append(URDU + i)
    
emotion = []
gender = []
path = []

for i in new_dir_list:
    fname = os.listdir(i)
    for f in fname:
        part = f.split('_')
        emotion.append(part[2][0])
        temp = part[0][1]
        if temp[0] == 'F':
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(i + '/' + f)

URUDU_df = pd.DataFrame(emotion, columns = ['labels'])
URUDU_df = URUDU_df.replace({'N':'neutral', 'H':'happy', 'S':'sad', 'A':'angry'})
URUDU_df = pd.concat([URUDU_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
URUDU_df['source'] = 'URUDU'  
URUDU_df = pd.concat([URUDU_df,pd.DataFrame(path, columns = ['path'])],axis=1)
URUDU_df.labels.value_counts()

angry      100
happy      100
neutral    100
sad        100
Name: labels, dtype: int64

<h3>JL dataset</h3>

<a href="https://github.com/tli725/JL-Corpus">JL link</a>

In [10]:
dir_list = os.listdir(JL)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]

emotion = []
gender = []
path = []
for f in dir_list:
    part = f.split('_')
    if part[3][4] == 't':
        continue
    if part[1] in ['assertive', 'apologetic', 'excited', 'concerned', 'encouraging', 'anxious']:
        continue
    emotion.append(part[1])
    temp = part[0][0]
    if temp == 'f':
        temp = "female"
    else:
        temp = "male"
    gender.append(temp)
    path.append(JL + f)
JL_df = pd.DataFrame(emotion, columns = ['labels'])
JL_df = pd.concat([JL_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
JL_df['source'] = 'JL'  
JL_df = pd.concat([JL_df,pd.DataFrame(path, columns = ['path'])],axis=1)
JL_df.labels.value_counts()

angry      240
happy      240
neutral    240
sad        240
Name: labels, dtype: int64

<h3>AESDD dataset</h3>

<a href="http://m3c.web.auth.gr/research/aesdd-speech-emotion-recognition/">AESDD link</a>

In [11]:
dir_list = os.listdir(AESDD)
dir_list.sort()
if '.DS_Store' in dir_list:
    dir_list = dir_list[1:]

emotion = []
gender = []
path = []
for f in dir_list:
    part = f.split(' ')
    emotion.append(part[0][0])
    temp = part[1][1]
    if int(temp) in [1, 2, 5]:
        temp = 'female'
    else:
        temp = 'male'
    gender.append(temp)
    path.append(AESDD + f)
    
AESDD_df = pd.DataFrame(emotion, columns = ['labels'])
AESDD_df = AESDD_df.replace({'h':'happy', 's':'sad', 'a':'angry', 'f':'fear', 'd':'disgust'})
AESDD_df = pd.concat([AESDD_df,pd.DataFrame(gender, columns = ['gender'])],axis=1)
AESDD_df['source'] = 'AESDD'  
AESDD_df = pd.concat([AESDD_df,pd.DataFrame(path, columns = ['path'])],axis=1)
AESDD_df.labels.value_counts()

sad        123
disgust    122
angry      121
fear       120
happy      119
Name: labels, dtype: int64

<h3>Concatenating datasets</h3>

In [25]:
'''# x language
df = pd.concat([SAVEE_df, RAV_df, TESS_df, JL_df, 
                CREMA_df, EMOVO_df, EMODB_df, 
                SHEMO_df, URUDU_df, AESDD_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)
#En
print("en")
df = pd.concat([SAVEE_df, RAV_df, TESS_df, JL_df, 
                CREMA_df], axis = 0)
df.head()
df.to_csv("Data_path.csv",index=False)

#ger
print("ger")
df = pd.concat([EMODB_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

#it
print("it")
df = pd.concat([EMOVO_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

#urd
print("urd")
df = pd.concat([URUDU_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

#pers
print("pers")
df = pd.concat([SHEMO_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)
'''
#greek
print("greek")
df = pd.concat([AESDD_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

ref = pd.read_csv("./Data_path.csv")
import soundfile as sf
import wave
import contextlib

def conta_tot():
    somma = 0
    for i in range(len(ref.path)):
        if i in [11866, 11867, 11868, 11869, 11870, 11871]:
            continue
        path = ref.path[i]
        if ref.labels[i] == 'surprise':
            continue
        try:
            y, sr = librosa.load(path, sr=16000, mono=True)
            somma += librosa.get_duration(y=y, sr=16000)

        except ValueError:
            print('file', row, 'non utilizzabile.')
    print("tot")
    print(somma)

def conta(st):
    somma = 0
    for i in range(len(ref.path)):
        if i in [11866, 11867, 11868, 11869, 11870, 11871]:
            continue
        path = ref.path[i]
        if ref.labels[i] != st:
            continue
        try:
            y, sr = librosa.load(path, sr=16000, mono=True)
            somma += librosa.get_duration(y=y, sr=16000)

        except ValueError:
            print('file', row, 'non utilizzabile.')
    print(st)
    print(somma)

conta_tot()
    
conta("angry")

conta("neutral")

conta("sad")

conta("happy")

conta("fear")

conta("disgust")

greek
sad        123
disgust    122
angry      121
fear       120
happy      119
Name: labels, dtype: int64
tot
2481.538312499999
angry
433.8998124999996
neutral
0
sad
562.77175
happy
478.8779375
fear
449.6054375
disgust
556.383375


In [ ]:
df = pd.concat([SAVEE_df, RAV_df, TESS_df, JL_df, 
                CREMA_df, EMOVO_df, EMODB_df, 
                SHEMO_df, URUDU_df, AESDD_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

In [19]:
df.count()

labels    18169
gender    18169
source    18169
path      18169
dtype: int64

In [20]:
ref = pd.read_csv("./Data_path.csv")

<h5>Converting all files in 16kHz Mono</h5>

<h4>creating dataset split in directories (representing the classes)</h4>

In [3]:
import soundfile as sf
for i in range(len(ref.path)):
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    path = ref.path[i]
    if ref.labels[i] == 'surprise':
        continue
    try:
        y, sr = librosa.load(path, sr=16000, mono=True)
        new = 'split_directories/all_data/' + ref.labels[i] + '/' + str(i) + '.wav'
        sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

In [4]:
import soundfile as sf
for i in range(len(ref.path)):
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    path = ref.path[i]
    if ref.labels[i] == 'surprise':
        continue
    try:
        if ref.gender[i] == 'female':
            y, sr = librosa.load(path, sr=16000, mono=True)
            new = 'split_directories/female_data/' + ref.labels[i] + '/' + str(i) + '.wav'
            sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

In [5]:
import soundfile as sf
for i in range(len(ref.path)):
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    path = ref.path[i]
    if ref.labels[i] == 'surprise':
        continue
    try:
        if ref.gender[i] == 'male':
            y, sr = librosa.load(path, sr=16000, mono=True)
            new = 'split_directories/male_data/' + ref.labels[i] + '/' + str(i) + '.wav'
            sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

In [6]:
import soundfile as sf
for i in range(len(ref.path)):
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    path = ref.path[i]
    if ref.labels[i] == 'surprise':
        continue
    try:
        y, sr = librosa.load(path, sr=16000, mono=True)
        if ref.gender[i] == 'male':
            new = 'split_directories/maleVsFemale/male/' + str(i) + '.wav'
            sf.write(new, y, sr)
        else:
            new = 'split_directories/maleVsFemale/female/' + str(i) + '.wav'
            sf.write(new, y, sr)
            
    except ValueError:
        print('file', row, 'non utilizzabile.')

<h4>creating dataset in single directory</h4>

In [3]:
import soundfile as sf
ref = pd.read_csv("./Data_path.csv")

for i in range(len(ref.path)):
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    path = ref.path[i]
    if ref.labels[i] == 'surprise':
        continue
    label = ref.labels[i]
    dic = {'neutral':'N', 'happy':'F', 'sad':'T', 'angry':'W', 'fear':'A', 'disgust':'E'}
    label = dic[label]
    module = str(i%10)
    st = str(i)
    try:
        y, sr = librosa.load(path, sr=16000, mono=True)
        new = 'dataset_x_net/' + st + label + module + '.wav'
        sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

In [5]:
import soundfile as sf
ref = pd.read_csv("./Data_path.csv")

for i in range(len(ref.path)):
    path = ref.path[i]
    # leaving out male data
    if ref.gender[i] == 'male' :
        continue
    if ref.labels[i] == 'surprise':
        continue
    label = ref.labels[i]
    dic = {'neutral':'N', 'happy':'F', 'sad':'T', 'angry':'W', 'fear':'A', 'disgust':'E'}
    label = dic[label]
    module = str(i%10)
    st = str(i)
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    try:
        y, sr = librosa.load(path, sr=16000, mono=True)
        new = 'dataset_x_net/' + st + label + module + '.wav'
        sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

In [6]:
import soundfile as sf
ref = pd.read_csv("./Data_path.csv")

for i in range(len(ref.path)):
    # leaving out female data
    if ref.gender[i] == 'female' :
        continue
    if i in [11866, 11867, 11868, 11869, 11870, 11871]:
        continue
    if ref.labels[i] == 'surprise':
        continue
    path = ref.path[i]
    label = ref.labels[i]
    dic = {'neutral':'N', 'happy':'F', 'sad':'T', 'angry':'W', 'fear':'A', 'disgust':'E'}
    label = dic[label]
    module = str(i%10)
    st = str(i)
    try:
        y, sr = librosa.load(path, sr=16000, mono=True)
        new = 'dataset_x_net/' + st + label + module + '.wav'
        sf.write(new, y, sr)
    except ValueError:
        print('file', row, 'non utilizzabile.')

<h2>Citations</h2>
<h5>SAVEE</h5>
<p>@inproceedings{Vlasenko_combiningframe,
author = {Vlasenko, Bogdan and Schuller, Bjorn and Wendemuth, Andreas and Rigoll, Gerhard},
year = {2007},
month = {01},
pages = {2249-2252},
title = {Combining frame and turn-level information for robust recognition of emotions within speech},
journal = {Proceedings of Interspeech}
}</p>

<h5>CREMA-D</h5>
<p>@article{cao2014crema,
  title={Crema-d: Crowd-sourced emotional multimodal actors dataset},
  author={Cao, Houwei and Cooper, David G and Keutmann, Michael K and Gur, Ruben C and Nenkova, Ani and Verma, Ragini},
  journal={IEEE transactions on affective computing},
  volume={5},
  number={4},
  pages={377--390},
  year={2014},
  publisher={IEEE}
}</p>

<h5>RAVEDESS</h5>
<p>@article{livingstone2018ryerson,
  title={The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS): A dynamic, multimodal set of facial and vocal expressions in North American English},
  author={Livingstone, Steven R and Russo, Frank A},
  journal={PloS one},
  volume={13},
  number={5},
  pages={e0196391},
  year={2018},
  publisher={Public Library of Science San Francisco, CA USA}
}</p>

<h5>TESS</h5>
<p>@data{SP2/E8H2MF_2020,
author = {Pichora-Fuller, M. Kathleen and Dupuis, Kate},
publisher = {Scholars Portal Dataverse},
title = {{Toronto emotional speech set (TESS)}},
year = {2020},
version = {DRAFT VERSION},
doi = {10.5683/SP2/E8H2MF},
url = {https://doi.org/10.5683/SP2/E8H2MF}
}</p>

<h5>EMOVO</h5>
<p>@inproceedings{costantini2014emovo,
  title={EMOVO corpus: an Italian emotional speech database},
  author={Costantini, Giovanni and Iaderola, Iacopo and Paoloni, Andrea and Todisco, Massimiliano},
  booktitle={International Conference on Language Resources and Evaluation (LREC 2014)},
  pages={3501--3504},
  year={2014},
  organization={European Language Resources Association (ELRA)}
}</p>

<h5>EmoDB</h5>
<p>@inproceedings{burkhardt2005database,
  title={A database of German emotional speech.},
  author={Burkhardt, Felix and Paeschke, Astrid and Rolfes, Miriam and Sendlmeier, Walter F and Weiss, Benjamin and others},
  booktitle={Interspeech},
  volume={5},
  pages={1517--1520},
  year={2005}
}</p>

<h5>ShEMO</h5>
<p>@article{nezami2019shemo,
  title={ShEMO: a large-scale validated database for Persian speech emotion detection},
  author={Nezami, Omid Mohamad and Lou, Paria Jamshid and Karami, Mansoureh},
  journal={Language Resources and Evaluation},
  volume={53},
  number={1},
  pages={1--16},
  year={2019},
  publisher={Springer}
}</p>

<h5>URDU</h5>
<p>@inproceedings{latif2018cross,
  title={Cross lingual speech emotion recognition: Urdu vs. western languages},
  author={Latif, Siddique and Qayyum, Adnan and Usman, Muhammad and Qadir, Junaid},
  booktitle={2018 International Conference on Frontiers of Information Technology (FIT)},
  pages={88--93},
  year={2018},
  organization={IEEE}
}</p>

<h5>JL</h5>
<p>@inproceedings{james2018open,
  title={An Open Source Emotional Speech Corpus for Human Robot Interaction Applications.},
  author={James, Jesin and Tian, Li and Watson, Catherine Inez},
  booktitle={INTERSPEECH},
  pages={2768--2772},
  year={2018}
}</p>

<h5>AESDD</h5>
<p>@article{vryzas2018speech,
  title={Speech emotion recognition for performance interaction},
  author={Vryzas, Nikolaos and Kotsakis, Rigas and Liatsou, Aikaterini and Dimoulas, Charalampos A and Kalliris, George},
  journal={Journal of the Audio Engineering Society},
  volume={66},
  number={6},
  pages={457--467},
  year={2018},
  publisher={Audio Engineering Society}
}</p>
<p>@incollection{vryzas2018subjective,
  title={Subjective Evaluation of a Speech Emotion Recognition Interaction Framework},
  author={Vryzas, Nikolaos and Matsiola, Maria and Kotsakis, Rigas and Dimoulas, Charalampos and Kalliris, George},
  booktitle={Proceedings of the Audio Mostly 2018 on Sound in Immersion and Emotion},
  pages={1--7},
  year={2018}
}</p>
